In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
#import sys
#!{sys.executable} -m pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import date
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from coinmetrics.api_client import CoinMetricsClient

In [3]:
halvings = ['2012-11-28', '2016-07-09', '2020-05-11', '2024-05-04']

In [4]:
#https://coinmetrics.io/introducing-the-revamped-coin-metrics-python-api-client/
from coinmetrics.api_client import CoinMetricsClient

gather_new_data = False
filename = 'data/on-chain-indicators.parquet'

try:
    table = pq.read_table(filename)
    metrics = table.to_pandas()
    last_date = metrics.index[-1].date()
    gather_new_data = last_date < date.today() - relativedelta(days = 1)
except FileNotFoundError:
    gather_new_data = True

if (gather_new_data):
    # Obtain metrics by specifying the asset, date interval and frequency.
    # All metrics available: https://docs.coinmetrics.io/info/metrics
    client = CoinMetricsClient()
    coinmetrics = client.get_asset_metrics(
        assets='btc',
        metrics=['PriceUSD',
                'AdrActCnt', #Addresses, active, count
                'AdrBalCnt', #Addresses, any balance, count
                'AdrBalNtv0.01Cnt', #Addresses, with balance, greater than 0.01 native units, count
                'AdrBalNtv0.1Cnt', #Addresses, with balance, greater than 0.1 native units, count
                'AdrBalNtv1Cnt', #Addresses, with balance, greater than 1 native units, count
                'AdrBalNtv10Cnt', #Addresses, with balance, greater than 10 native units, count
                'AdrBalNtv100Cnt', #Addresses, with balance, greater than 100 native units, count
                'AdrBalNtv1KCnt', #Addresses, with balance, greater than 1000 native units, count
                'AdrBalNtv10KCnt', #Addresses, with balance, greater than 10000 native units, count
                'CapMrktCurUSD', #Capitalization, market, current supply, USD
                'CapMVRVCur', #Capitalization, MVRV, current supply
                'CapRealUSD', #Capitalization, realized, USD
                'NVTAdj', #NVT, adjusted
                'NVTAdj90', #NVT, adjusted, 90d MA
                'HashRate', #Hash rate, mean
                'TxTfrValAdjUSD' #Transactions, transfers, value, adjusted, USD
                ],
        start_time=last_date,
        #end_time='2021-12-17',
        frequency='1d'
        )
    coinmetrics = pd.DataFrame(coinmetrics)
    coinmetrics['time'] = pd.to_datetime(coinmetrics['time'])
    coinmetrics = coinmetrics.set_index('time')
    coinmetrics['PriceUSD'] = pd.to_numeric(coinmetrics['PriceUSD'])
    coinmetrics['AdrActCnt'] = pd.to_numeric(coinmetrics['AdrActCnt'])
    coinmetrics['AdrBalCnt'] = pd.to_numeric(coinmetrics['AdrBalCnt'])
    coinmetrics['AdrBalNtv0.01Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv0.01Cnt'])
    coinmetrics['AdrBalNtv0.1Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv0.1Cnt'])
    coinmetrics['AdrBalNtv1Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv1Cnt'])
    coinmetrics['AdrBalNtv10Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv10Cnt'])
    coinmetrics['AdrBalNtv100Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv100Cnt'])
    coinmetrics['AdrBalNtv1KCnt'] = pd.to_numeric(coinmetrics['AdrBalNtv1KCnt'])
    coinmetrics['AdrBalNtv10KCnt'] = pd.to_numeric(coinmetrics['AdrBalNtv10KCnt'])
    coinmetrics['CapMrktCurUSD'] = pd.to_numeric(coinmetrics['CapMrktCurUSD'])
    coinmetrics['CapRealUSD'] = pd.to_numeric(coinmetrics['CapRealUSD'])
    coinmetrics['CapMVRVCur'] = pd.to_numeric(coinmetrics['CapMVRVCur'])
    coinmetrics['NVTAdj'] = pd.to_numeric(coinmetrics['NVTAdj'])
    coinmetrics['NVTAdj90'] = pd.to_numeric(coinmetrics['NVTAdj90'])
    coinmetrics['HashRate'] = pd.to_numeric(coinmetrics['HashRate'])
    coinmetrics['TxTfrValAdjUSD'] = pd.to_numeric(coinmetrics['TxTfrValAdjUSD'])

    metrics = pd.concat([metrics, coinmetrics])
    metrics.drop_duplicates(inplace=True)

table = pa.Table.from_pandas(metrics)
pq.write_table(table, filename)


In [11]:
#MVRV
import plotly.graph_objects as go
from plotly.subplots import make_subplots

mvrv_data = metrics[['PriceUSD', 'CapMVRVCur', 'CapMrktCurUSD', 'CapRealUSD']].copy()
#mvrv_data['Z_Score'] = (mvrv_data['CapMrktCurUSD'] - mvrv_data['CapRealUSD']) / mvrv_data['CapMrktCurUSD'].expanding().std(ddof=0)
mvrv_data = mvrv_data['2012-11-28':]
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=mvrv_data.index, y=mvrv_data['PriceUSD'], name='Price'),
        secondary_y=True)
fig.add_trace(
    go.Scatter(x=mvrv_data.index, y=mvrv_data['CapMVRVCur'], name='MVRV'),
        secondary_y=False)
fig.add_hrect(y0=3.9, y1=6.0, line_width=0, fillcolor='red', opacity=0.2, secondary_y=False)
fig.add_hrect(y0=0.0, y1=0.8, line_width=0, fillcolor='green', opacity=0.2, secondary_y=False)

# Add figure title
fig.update_layout(
    title_text='MVRV Ratio',
    legend=dict(orientation="h"),
    width=1400, height=700,
    hovermode='x unified')

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='MVRV', secondary_y=False)

fig.show()

In [12]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name='Price'),
        secondary_y=False)

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['AdrBalCnt'], name='Number of Addresses'),
        secondary_y=True)

# Add figure title
fig.update_layout(
    title_text='Number of Addresses with a Non-Zero Balance',
    legend=dict(orientation="h"))

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=False)
fig.update_yaxes(title_text='Addresses with a Non-Zero Balance', secondary_y=True)

fig.show()

In [13]:
# Bitcoin Realized Cap
# Colorscales: https://plotly.com/python/builtin-colorscales/
data = metrics['2011-01-01':].copy()
data['Distance'] = (np.log(data['CapMrktCurUSD']) - np.log(data['CapRealUSD']))

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, 
                         y=data['CapMrktCurUSD'],
                         name='Market Cap',
                         mode='markers',
                         marker=dict(
                            size=6,
                            color=data['Distance'], # set color equal to a variable
                            colorscale='Rainbow',
                            colorbar=dict(
                                title='Distance between Market and Realized Cap',
                                titleside='right'
                            ),
                            showscale=True)
                        ))
fig.add_trace(go.Scatter(x=data.index, 
                         y=data['CapRealUSD'],
                         name='Realized Cap',
                         mode='lines'))

fig.update_yaxes(type="log")

fig.update_layout(title='Bitcoin Realized Cap',
                  legend=dict(orientation='h'),
                  width=1400, height=600)

fig.show()

In [14]:
#NUPL
import plotly.graph_objects as go
from plotly.subplots import make_subplots

nupl_data = metrics['2012-11-28':][['PriceUSD', 'CapMrktCurUSD', 'CapRealUSD']]
nupl_data['nupl'] = (nupl_data['CapMrktCurUSD'] - nupl_data['CapRealUSD']) / nupl_data['CapMrktCurUSD']
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=nupl_data.index, y=nupl_data['PriceUSD'], name='Price'),
        secondary_y=True)
fig.add_trace(
    go.Scatter(x=nupl_data.index, y=nupl_data['nupl'], name='NUPL'),
        secondary_y=False)

#Capitulation
fig.add_hrect(y0=-1.0, y1=0.0,
    line_width=0, fillcolor='blue',
    opacity=0.3,
    annotation_text="Capitulation", annotation_position="top left",
    secondary_y=False)

#Hope/Fear
fig.add_hrect(y0=0.0, y1=0.25,
    line_width=0,
    fillcolor='green',
    opacity=0.3,
    annotation_text="Hope / Fear", annotation_position="top left",
    secondary_y=False)

#Optimism/Anxiety
fig.add_hrect(y0=0.25, y1=0.50,
    line_width=0,
    fillcolor='yellow',
    opacity=0.3,
    annotation_text="Optimism / Anxiety", annotation_position="top left",
    secondary_y=False)

#Belief/Denial
fig.add_hrect(y0=0.50, y1=0.75,
    line_width=0,
    fillcolor='orange',
    opacity=0.3,
    annotation_text="Belief / Denial",
    annotation_position="top left",
    secondary_y=False)

#Euphoria/Greed
fig.add_hrect(y0=0.75, y1=1.0,
    line_width=0,
    fillcolor='red',
    opacity=0.3,
    annotation_text="Euphoria / Greed", annotation_position="top left",
    secondary_y=False)

# Add figure title
fig.update_layout(
    title_text='Net Unrealized Profit/Loss (NUPL)',
    legend=dict(orientation="h"),
    width=1400, height=800,
    hovermode='x unified')

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='NUPL', secondary_y=False)

fig.show()

In [15]:
# Bitcoin Price - NUPL
time_period = '1W'
nupl_data = metrics['2012-11-28':][['PriceUSD', 'CapMrktCurUSD', 'CapRealUSD']].resample(time_period).last()
nupl_data['nupl'] = (nupl_data['CapMrktCurUSD'] - nupl_data['CapRealUSD']) / nupl_data['CapMrktCurUSD']

nupl_data['nupl_range'] = 0 # Capitulation
nupl_data['nupl_range'] = np.where((nupl_data['nupl'] > 0.0) & (nupl_data['nupl'] <= 0.25), 1, nupl_data['nupl_range']) # Hope/Fear
nupl_data['nupl_range'] = np.where((nupl_data['nupl'] > 0.25) & (nupl_data['nupl'] <= 0.50), 2, nupl_data['nupl_range']) # Optimism/Anxiety
nupl_data['nupl_range'] = np.where((nupl_data['nupl'] > 0.50) & (nupl_data['nupl'] <= 0.75), 3, nupl_data['nupl_range']) # Belief/Denial
nupl_data['nupl_range'] = np.where(nupl_data['nupl'] > 0.75, 4, nupl_data['nupl_range']) # Euphoria/Greed

fig = go.Figure()
fig.add_trace(go.Scatter(x=nupl_data.index,
                         y=nupl_data['PriceUSD'],
                         name='Price',
                         mode='markers',
                         marker=dict(
                            size=8,
                            color=nupl_data['nupl_range'], # set color equal to a variable
                            colorscale='jet',
                            colorbar=dict(
                                title="NUPL",
                                titleside="right",
                                tickmode="array",
                                tickvals=[0, 1, 2, 3, 4],
                                ticktext=["Capitulation", "Hope/Fear", "Optimism/Anxiety", "Belief/Denial", "Euphoria/Greed"],
                                ticks="outside"
                            )
                        )
                    )
)

fig.update_yaxes(type="log")

fig.update_layout(title='Bitcoin Price / NUPL',
                  width=1200, height=600)

fig.show()

In [16]:
import yfinance as yf
import math

# interval '1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo'
spy_ohlc_df = yf.download('SPY', start='2009-01-01', interval='1d')
spy_ohlc_df = spy_ohlc_df.dropna()

[*********************100%%**********************]  1 of 1 completed


In [19]:
data_spy = spy_ohlc_df['2015-01-01':].copy()
data_btc = metrics['2015-01-01':].copy()
# normalize data
data_spy['Adj Close'] = np.log(data_spy['Adj Close'])
data_btc['PriceUSD'] = np.log(data_btc['PriceUSD'])
data_spy['Adj Close'] = (data_spy['Adj Close'] - data_spy['Adj Close'].mean()) / data_spy['Adj Close'].std()
data_btc['PriceUSD'] = (data_btc['PriceUSD'] - data_btc['PriceUSD'].mean()) / data_btc['PriceUSD'].std()

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_spy.index, y=data_spy['Adj Close'],
                    mode='lines',
                    name='S&P500'))
fig.add_trace(go.Scatter(x=data_btc.index, y=data_btc['PriceUSD'],
                    mode='lines',
                    name='BTC'))

fig.show()